In [13]:
%%capture
!uv pip install s3fs zarr ctime fsspec

*Workshop 4. Doing research with hydrological data*


# Practical 1: Uncertainty in rainfall estimates

## Contents
- Load in rain gauge and grid data
- Remove rain gauge data from dodgy gauges
- Compare estimations of events, and the differences


## Objectives:
- Understand the uncertainties in rainfall estimates between gridded rainfall and rain gauge data
-


# Introduction
Rainfall is famously difficult to measure/observe. But the best appraoches we currently use are rain gauges, rain radar. Satelitte-derived precipitation is much less accurate  

[Map of UK Gauge station]  

Also, a basic [interactive map](https://thomasjkeel.github.io/UK-Rain-Gauge-Network/gauges.html)  

We will be using the CEH-GEAR dataset for UK rainfall, but there is also the HadUK-Grid (link), which is more common, but uses a more uncertain spatial interpolation method.

> *Gridded rainfall products provide a best guess estimate of rainfall*

[image of gridding]  


In [30]:
# Load required libraries
import fsspec
import zarr

import pandas as pd
import polars as pl
import xarray as xr

import matplotlib.pyplot as plt

## 1. Load data

**Data sources:**
- rain gauge and gridded rainfall data - from JASMIN object-store

In [15]:
fdri_fs = fsspec.filesystem("s3", asynchronous=True, anon=True, endpoint_url="https://fdri-o.s3-ext.jc.rl.ac.uk")
gear_daily_zstore = zarr.storage.FsspecStore(fdri_fs, path="geardaily/GB/geardaily_fulloutput_yearly_100km_chunks.zarr")
gear_daily = xr.open_zarr(gear_daily_zstore, decode_times=True, decode_cf=True)
gear_daily

<xarray.Dataset> Size: 666GB
Dimensions:          (y: 1251, x: 701, time: 47481)
Coordinates:
    crs              int16 2B ...
    lat              (y, x) float64 7MB dask.array<chunksize=(100, 100), meta=np.ndarray>
    lon              (y, x) float64 7MB dask.array<chunksize=(100, 100), meta=np.ndarray>
  * time             (time) datetime64[ns] 380kB 1890-01-01 ... 2019-12-31
  * x                (x) float64 6kB 0.0 1e+03 2e+03 ... 6.98e+05 6.99e+05 7e+05
  * y                (y) float64 10kB 1.25e+06 1.249e+06 1.248e+06 ... 1e+03 0.0
Data variables:
    min_dist         (time, y, x) float64 333GB dask.array<chunksize=(365, 100, 100), meta=np.ndarray>
    rainfall_amount  (time, y, x) float64 333GB dask.array<chunksize=(365, 100, 100), meta=np.ndarray>
Attributes: (12/21)
    Conventions:         CF-1.6
    comment:             In line with standard UK convention, the rainfall on...
    creator_email:       malngu@ceh.ac.uk, vke@ceh.ac.uk
    creator_name:        Maliko Tanguy, Virginie Keller
    creator_url:         
    date_issued:         2014-01-06
    ...                  ...
    publisher_email:     enquiries@ceh.ac.uk
    publisher_name:      Centre for Ecology and Hydrology
    publisher_url:       http://www.ceh.ac.uk
    reference:           Tanguy, M., Prosdocimi, I., Keller, V.D.J., Terry, J...
    source:              CEH - GEAR: dataset created using Natural Neighbour ...
    version_comment:     The version number of each data file corresponds to ...

In [9]:
severn_rain_gauge_data = pd.read_csv('s3://rain-gauge/hourly_severn_rain_gauge_data.csv', storage_options={'endpoint_url': "https://fdri-o.s3-ext.jc.rl.ac.uk", 'anon': True})
severn_rain_gauge_metadata = pd.read_csv('s3://rain-gauge/hourly_severn_rain_gauge_metadata.csv', storage_options={'endpoint_url': "https://fdri-o.s3-ext.jc.rl.ac.uk", 'anon': True})

In [33]:
severn_rain_gauge_data = pl.read_csv('s3://rain-gauge/hourly_severn_rain_gauge_data.csv', storage_options={'endpoint_url': "https://fdri-o.s3-ext.jc.rl.ac.uk", 'anon': True}, try_parse_dates=True)


In [34]:
severn_rain_gauge_data

ID,DATETIME,PRECIPITATION
i64,datetime[μs],f64
89714,1978-10-01 00:00:00,0.0
89714,1978-10-02 00:00:00,2.0
89714,1978-10-03 00:00:00,0.0
89714,1978-10-04 00:00:00,0.0
89714,1978-10-05 00:00:00,0.0
…,…,…
429880,1965-10-19 00:00:00,0.0
429880,1965-10-20 00:00:00,0.0
429880,1965-10-21 00:00:00,0.0


In [10]:
# format datetime
severn_rain_gauge_data['DATETIME'] = pd.to_datetime(severn_rain_gauge_data['DATETIME'], format="mixed")

In [25]:
severn_rain_gauge_data.head()

,ID,DATETIME,PRECIPITATION
0,89714,1978-01-10,0.0
1,89714,1978-02-10,2.0
2,89714,1978-03-10,0.0
3,89714,1978-04-10,0.0
4,89714,1978-05-10,0.0


In [21]:
severn_rain_gauge_metadata.head()

,ID,SRC_ID,NAME,COUNTRY_CODE,EASTING,NORTHING,HYDROMETRIC_AREA,ELEVATION,GEOG_PATH
0,89714,2913,STRONGFORD W WKS,GB-GBN,387932,339157,28,95,/BI/UK/GB/ENG/STS/
1,90358,2918,SUGNALL HALL,GB-GBN,379831,331185,28,143,/BI/UK/GB/ENG/STS/
2,90359,2919,"ECCLESHALL, SUGNALL HALL",GB-GBN,379800,331200,28,137,/BI/UK/GB/ENG/STS/
3,90492,2920,WALTON HALL GARDENS,GB-GBN,384900,328500,28,99,/BI/UK/GB/ENG/STS/
4,90537,2921,WHITMORE P STA,GB-GBN,379900,340100,28,121,/BI/UK/GB/ENG/CHS/


# 2. Format the data
Most of data science is about data cleaning and formatting. This is especially true for those using environmental data

### 2.1 Quality controlling rain gauge dataset
Data from the rain gauge data is not Quality-controlled


In [29]:
severn_rain_gauge_data.loc[severn_rain_gauge_data["ID"] == 90537]

,ID,DATETIME,PRECIPITATION
37467,90537,1961-01-01,2.0
37468,90537,1961-02-01,5.1
37469,90537,1961-03-01,1.5
37470,90537,1961-04-01,0.0
37471,90537,1961-05-01,10.9
...,...,...,...
47567,90537,1989-03-27,7.3
47568,90537,1989-03-28,0.1
47569,90537,1989-03-29,0.0
47570,90537,1989-03-30,0.0


# 3. Compare gridded rainfall product (CEH-GEAR) with rain gauge data

## Case Study: Carreg Wen

In [12]:
carreg_daily_new = pl.read_csv('../gauge_data/plynlimon-jwk-2024-infilled_carreg_rf.csv', skip_lines=13, columns=[0, 1], new_columns=['time', f'{RAIN_COL}_gauge_new'], try_parse_dates=True)
carreg_daily_new.head()

#### 🤨 Tasks

???
Replace the ??? below with your answer

## ❗❗ FURTHER TASKS ❗❗  
Feel free to stop at this point, but below are some additional and more advanced topics and tasks requiring more of your own input. We will provide help.

Task 1. Look at another catchment  
Task 2. Use CHESS-SCAPE data  
Task 3. How does...  


## Extra task: using CHESS-SCAPE climate projection data for precipitation and temperature

In [ ]:
# We are accessing TASMAX & PRCPT for the Ensemble member #01 from the catalogue
fs = fsspec.filesystem("s3", asynchronous=True, anon=True, endpoint_url="https://chess-scape-o.s3-ext.jc.rl.ac.uk")
tmax_zstore = zarr.storage.FsspecStore(fs, path="ens01-year100kmchunk/tmax_01_year100km.zarr")
pr_zstore = zarr.storage.FsspecStore(fs, path="ens01-year100kmchunk/pr_01_year100km.zarr")

chess_tmax = xr.open_zarr(tmax_zstore, decode_times=True, decode_cf=True, consolidated=False)
chess_pr = xr.open_zarr(pr_zstore, decode_times=True, decode_cf=True, consolidated=False)

# Additional Reading

- <https://github.com/NERC-CEH/FDRI-comparing-rainfall-data-in-upper-severn/tree/main>  

- <https://github.com/NERC-CEH/FDRI-high-altitude-rainfall-and-floods>